In [3]:
pip install surprise

     |████████████████████████████████| 11.8MB 14.9MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp36-cp36m-linux_x86_64.whl size=1618303 sha256=ac08ef8a5f5522ff288b075e5873973e4f22d65b3b9bc10b8cf57ce81a3d4b3d
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [48]:
import pandas as pd
import glob
from google.colab import drive
from surprise import Dataset,Reader
from surprise.model_selection import train_test_split
from surprise import SVD
from surprise import KNNWithMeans
from surprise import accuracy
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Merging CSV into one Dataframe : 

In [2]:
path = r'/content/drive/MyDrive/GL- AIML/Recommendation System/Dataset'
all_files = glob.glob(path + "/*.csv")
df = pd.concat((pd.read_csv(f,encoding = 'latin-1') for f in all_files))
df.shape

(1415133, 11)

Sample of Dataframe Created : 

In [3]:
df.sample(5)

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
237174,/cellphones/asus-ze550kl/,5/19/2016,en,in,Amazon,amazon.in,8.0,10.0,Amzon delevry on time. but paking is very very...,anoj,Asus Zenfone 2 Laser 5.5 ZE550KL-1B122IN (White)
269271,/cellphones/apple-iphone-6/,11/9/2014,fr,fr,Amazon,amazon.fr,10.0,10.0,TrÃ¨s bon produit exceptionnelle rien a dire l...,LAURENT JOIGNY,Apple iPhone 6 Smartphone dÃ©bloquÃ© 4G (Ecran...
6556,/cellphones/samsung-galaxy-s7-edge/,11/11/2016,en,us,Samsung,samsung.com,9.2,10.0,Overall I love my phone and its perfect for wh...,Lmurphy,Samsung Galaxy S7 edge 32GB (Verizon)
155793,/cellphones/xiaomi-mi-max/,7/17/2016,en,in,Amazon,amazon.in,10.0,10.0,The best phone ever used . Amazing phone with ...,Amazon Customer,"Mi Xiaomi Mi Max (Gold, 32GB)"
72720,/cellphones/lg-google-nexus-5/,2/17/2014,fr,fr,Amazon,amazon.fr,8.0,10.0,"Assez puissant, Android bien, hergonomique, bo...",Client d'Amazon,LG Nexus 5 Smartphone dÃ©bloquÃ© 4G (Ecran: 5 ...


Here we can see that there are reviews in certain different languages.

5 point summary for the dataframe : 

In [4]:
df.describe(include = 'all')

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
count,1415133,1415133,1415133,1415133,1415133,1415133,1.351644e+06,1351644.0,1395772,1351931,1415132
unique,5556,7728,22,42,331,384,NaN,NaN,1321353,801103,61313
top,/cellphones/samsung-galaxy-s-iii/,7/18/2016,en,us,Amazon,amazon.com,NaN,NaN,#NAME?,Amazon Customer,"Lenovo Vibe K4 Note (White,16GB)"
freq,17093,3244,554746,318435,728471,214776,NaN,NaN,667,76978,5226
mean,NaN,NaN,NaN,NaN,NaN,NaN,8.007060e+00,10.0,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,2.616121e+00,0.0,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,2.000000e-01,10.0,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,7.200000e+00,10.0,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,9.200000e+00,10.0,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e+01,10.0,NaN,NaN,NaN


There are only two values which are numerical.
Score Max consists of all values as 10.

1. Top product is Lenovo Vibe K4
2. Top author is Amazon Customer



Datatypes for the columns : 

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1415133 entries, 0 to 163836
Data columns (total 11 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   phone_url  1415133 non-null  object 
 1   date       1415133 non-null  object 
 2   lang       1415133 non-null  object 
 3   country    1415133 non-null  object 
 4   source     1415133 non-null  object 
 5   domain     1415133 non-null  object 
 6   score      1351644 non-null  float64
 7   score_max  1351644 non-null  float64
 8   extract    1395772 non-null  object 
 9   author     1351931 non-null  object 
 10  product    1415132 non-null  object 
dtypes: float64(2), object(9)
memory usage: 129.6+ MB


There are only two columns which have float values else all are object datatype.

Unique Values in each column : 

In [6]:
df.nunique()

phone_url       5556
date            7728
lang              22
country           42
source           331
domain           384
score             86
score_max          1
extract      1321353
author        801103
product        61313
dtype: int64

As we can see there is only one value in score_max. Our asssumption stands correct.

1.   There are 42 unique countries and 22 unique languages.
2.   There are 801103 unique authors and 61313 unique products.



Rounding off Score values :

In [7]:
df['score'] = df['score'].round(decimals =0)

In [8]:
df.head()

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Verizon Wireless,verizonwireless.com,10.0,10.0,As a diehard Samsung fan who has had every Sam...,CarolAnn35,Samsung Galaxy S8
1,/cellphones/samsung-galaxy-s8/,4/28/2017,en,us,Phone Arena,phonearena.com,10.0,10.0,Love the phone. the phone is sleek and smooth ...,james0923,Samsung Galaxy S8
2,/cellphones/samsung-galaxy-s8/,5/4/2017,en,us,Amazon,amazon.com,6.0,10.0,Adequate feel. Nice heft. Processor's still sl...,R. Craig,"Samsung Galaxy S8 (64GB) G950U 5.8"" 4G LTE Unl..."
3,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Samsung,samsung.com,9.0,10.0,Never disappointed. One of the reasons I've be...,Buster2020,Samsung Galaxy S8 64GB (AT&T)
4,/cellphones/samsung-galaxy-s8/,5/11/2017,en,us,Verizon Wireless,verizonwireless.com,4.0,10.0,I've now found that i'm in a group of people t...,S Ate Mine,Samsung Galaxy S8


Checking Null Values : 

In [9]:
df.isnull().sum()

phone_url        0
date             0
lang             0
country          0
source           0
domain           0
score        63489
score_max    63489
extract      19361
author       63202
product          1
dtype: int64



1.   We have null values in 5 columns, score and score max is important, we fill first impute them.
2.   Extract and Author column also has a considerable amount of null records, we will treat them too separately.



Treating Null Values for Score and Score_Max Column :

In [10]:
df['score'].fillna(df['score'].mean(),axis=0,inplace=True)
df['score'] = df['score'].round(decimals =0)
df['score_max'].fillna(df['score_max'].median(),axis=0,inplace=True)

In [11]:
df.isnull().sum()

phone_url        0
date             0
lang             0
country          0
source           0
domain           0
score            0
score_max        0
extract      19361
author       63202
product          1
dtype: int64

Building Function for treating categorical column null values based on Country :

In [12]:
def impute_categorical(categorical_column1, categorical_column2):
    cat_frames = []
    for i in list(set(df[categorical_column1])):
        df_category = df[df[categorical_column1]== i]
        if len(df_category) > 1:    
            df_category[categorical_column2].fillna(df_category[categorical_column2].mode()[0],inplace = True)        
        else:
            df_category[categorical_column2].fillna(df[categorical_column2].mode()[0],inplace = True)
        cat_frames.append(df_category)    
        cat_df = pd.concat(cat_frames)
    return cat_df

df = impute_categorical('country', 'author')
df = impute_categorical('country', 'extract')


Checking Shape of imputed dataframe :

In [13]:
print(df.isnull().sum())
print('Shape of dataframe : ',df.shape)

phone_url    0
date         0
lang         0
country      0
source       0
domain       0
score        0
score_max    0
extract      0
author       0
product      1
dtype: int64
Shape of dataframe :  (1415133, 11)


Now we can see there is no null value present in our dataset, only 1 null value is there in product column.
First we will try to drop duplicates if that solves our 1 null product problem

Checking Duplicate Rows :

In [14]:
df[df.duplicated(keep = 'first')]

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
4488,/cellphones/samsung-galaxy-s6-edgeplus/,10/27/2016,it,it,Amazon,amazon.it,8.0,10.0,â® Recensione smartphone Samsung S7 Edge â® ...,VarryX Reviews,"Samsung Galaxy S6 edge+ Smartphone 32 GB, Nero..."
16818,/cellphones/samsung-galaxy-s7-edge/,10/27/2016,it,it,Amazon,amazon.it,8.0,10.0,â® Recensione smartphone Samsung S7 Edge â® ...,VarryX Reviews,Samsung G935 Galaxy S7 Edge Smartphone da 32GB...
16824,/cellphones/samsung-galaxy-s7-edge/,10/27/2016,it,it,Amazon,amazon.it,8.0,10.0,â® Recensione smartphone Samsung S7 Edge â® ...,VarryX Reviews,Samsung G935 Galaxy S7 Edge Smartphone da 32GB...
16826,/cellphones/samsung-galaxy-s7-edge/,10/27/2016,it,it,Amazon,amazon.it,8.0,10.0,â® Recensione smartphone Samsung S7 Edge â® ...,VarryX Reviews,Samsung Galaxy S7 edge 32GB Pink
16833,/cellphones/samsung-galaxy-s7-edge/,10/27/2016,it,it,Amazon,amazon.it,8.0,10.0,â® Recensione smartphone Samsung S7 Edge â® ...,VarryX Reviews,Samsung G935 Galaxy S7 Edge Smartphone da 32 G...
...,...,...,...,...,...,...,...,...,...,...,...
59238,/cellphones/xiaomi-redmi-1s/,9/4/2014,pt,pt,Efox,efox.com.pt,10.0,10.0,"OlÃ¡ , fiz uma encomenda com o nÂº 20140912790...",HMD11271,XIAOMI HONGMI REDMI 1S MSM8228 1.6GHz 4.7 Pole...
32026,/cellphones/samsung-galaxy-note/,20/11/2011,pt,pt,Fnac,fnac.pt,10.0,10.0,"Para um publico especifico, que ande sempre co...",Hugo Alencastre,Samsung Galaxy Note 16GB N7000 (Black)
21025,/cellphones/samsung-galaxy-s-plus/,9/12/2012,pt,pt,Samsung,samsung.com,6.0,10.0,venho reclamar aqui pois tenho um samsung s pl...,Leandro Pereira,Samsung Galaxy S Plus
21203,/cellphones/samsung-galaxy-s-plus/,4/3/2012,pt,pt,Samsung,samsung.com,6.0,10.0,Leiam bem alguns foruns antes de comprar este ...,Lopes,Samsung Galaxy S Plus


Dropping Duplicate Rows :

In [15]:
df = df.drop_duplicates(keep = 'first')

In [16]:
df[df.duplicated(keep = 'last')]

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product


As there is only 1 null value in product column, dropping the whole row for the same :

In [17]:
df = df[df['product'].notna()]
print(df.isnull().sum())
print('Shape of dataframe : ',df.shape)

phone_url    0
date         0
lang         0
country      0
source       0
domain       0
score        0
score_max    0
extract      0
author       0
product      0
dtype: int64
Shape of dataframe :  (1408690, 11)


Now our dataframe does not have any null values in it, all are imputed.

Keeping only 1000000 values with random_state set as 612, resetting the index :

In [18]:
df = df.sample(n = 1000000 , random_state = 612)
df.reset_index(inplace= True)

Checking shape of & Null values in Dataframe :

In [19]:
df.shape

(1000000, 12)

In [20]:
print(df.isnull().sum())


index        0
phone_url    0
date         0
lang         0
country      0
source       0
domain       0
score        0
score_max    0
extract      0
author       0
product      0
dtype: int64


In [21]:
df.head(5)

,index,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,234756,/cellphones/asus-zenfone-2-ze551ml/,12/29/2015,pt,br,Kabum,kabum.com.br,10.0,10.0,Muito bom,AnÃ´nimo,Smartphone Asus Zenfone 2 ZE551ML - 6C542WW c/...
1,176849,/cellphones/huawei-p8-lite/,5/26/2016,fi,fi,Verkkokauppa,verkkokauppa.com,8.0,10.0,Alle 200â¬:n Ã¤lypuhelimeksi Huawei P8 Lite o...,Tekido81,"Huawei P8 Lite Dual-SIM âAndroid-puhelin, musta"
2,340453,/cellphones/nokia-6300/,6/25/2008,en,gb,Amazon,amazon.co.uk,4.0,10.0,I purchased this phone at the end of December ...,Peter Roberts,Nokia 6300 - All Black Edition (Unlocked to al...
3,90984,/cellphones/samsung-galaxy-s5-mini/,8/5/2014,es,es,Amazon,amazon.es,10.0,10.0,Perfecto para lo que yo necesito. Se echa en f...,Cliente Amazon,Samsung Galaxy S5 Mini - Smartphone libre Andr...
4,153919,/cellphones/samsung-e1150/,2/4/2014,de,de,Amazon,amazon.de,10.0,10.0,"Ich habe dieses Mobile Phone zweimal gekauft, ...",BÃ¼chermania,"Samsung E1150i Klapphandy 3,6 cm (1,43 Zoll) D..."


Dropping un-necessary columns and columns which contains information which is present in other columns : 

In [22]:
df = df.drop(['index','phone_url','domain','score_max'],axis =1 )
df.head(5)

,date,lang,country,source,score,extract,author,product
0,12/29/2015,pt,br,Kabum,10.0,Muito bom,AnÃ´nimo,Smartphone Asus Zenfone 2 ZE551ML - 6C542WW c/...
1,5/26/2016,fi,fi,Verkkokauppa,8.0,Alle 200â¬:n Ã¤lypuhelimeksi Huawei P8 Lite o...,Tekido81,"Huawei P8 Lite Dual-SIM âAndroid-puhelin, musta"
2,6/25/2008,en,gb,Amazon,4.0,I purchased this phone at the end of December ...,Peter Roberts,Nokia 6300 - All Black Edition (Unlocked to al...
3,8/5/2014,es,es,Amazon,10.0,Perfecto para lo que yo necesito. Se echa en f...,Cliente Amazon,Samsung Galaxy S5 Mini - Smartphone libre Andr...
4,2/4/2014,de,de,Amazon,10.0,"Ich habe dieses Mobile Phone zweimal gekauft, ...",BÃ¼chermania,"Samsung E1150i Klapphandy 3,6 cm (1,43 Zoll) D..."


Identifying the most rated product :

In [23]:
df.groupby("product")["score"].count().sort_values(ascending=False).head(10)

product
Lenovo Vibe K4 Note (White,16GB)       3720
Lenovo Vibe K4 Note (Black, 16GB)      3098
OnePlus 3 (Graphite, 64 GB)            2924
OnePlus 3 (Soft Gold, 64 GB)           2470
Samsung Galaxy Express I8730           1932
Huawei P8lite zwart / 16 GB            1914
Lenovo Vibe K5 (Gold, VoLTE update)    1784
Samsung Galaxy S6 zwart / 32 GB        1673
Lenovo Vibe K5 (Grey, VoLTE update)    1510
Nokia 5800 XpressMusic                 1490
Name: score, dtype: int64

These are the top 10 phones (max reviewed)

Identifying author with most reviews :

In [24]:
df.groupby("author")["score"].count().sort_values(ascending=False).head(10)

author
Amazon Customer       56141
ÐÐ»ÐµÐºÑÐ°Ð½Ð´Ñ    29279
Cliente Amazon        14933
Amazon Kunde           7046
e-bit                  6379
Client d'Amazon        6129
ÐÐ¾ÑÑÑ             5595
unknown                2164
Anonymous              1974
einer Kundin           1836
Name: score, dtype: int64

These are the top 10 authors (max reviews)

Selecting authors with 50 or more than 50 reviews :

In [25]:
counts = df['author'].value_counts()
df_a = df[df['author'].isin(counts[counts>=50].index)]
df_a

,date,lang,country,source,score,extract,author,product
0,12/29/2015,pt,br,Kabum,10.0,Muito bom,AnÃ´nimo,Smartphone Asus Zenfone 2 ZE551ML - 6C542WW c/...
3,8/5/2014,es,es,Amazon,10.0,Perfecto para lo que yo necesito. Se echa en f...,Cliente Amazon,Samsung Galaxy S5 Mini - Smartphone libre Andr...
5,11/4/2013,ru,ru,Yandex,2.0,?????????????????? ??????????? ????????? ???...,ÐÐ»ÐµÐºÑÐ°Ð½Ð´Ñ,HTC Sensation XE
7,10/19/2016,en,us,Amazon,2.0,Do NOT buy this phone! You are better off gett...,Leo,"BLU Vivo XL Smartphone - 5.5"" 4G LTE - GSM Unl..."
9,9/23/2015,de,de,Amazon,6.0,Hab es jetzt so ca 1.5 Jahre. Muss mir allerdi...,Florian,"Samsung Galaxy S4 Smartphone (12,7 cm (4,9 Zol..."
...,...,...,...,...,...,...,...,...
999988,11/30/2016,de,de,Otto.de,10.0,Dieses Smartphone erfÃ¼llt alle meine Erwartun...,einem Kunden,"Samsung Galaxy S7 edge Smartphone, 13,9 cm (5,..."
999991,12/30/2015,en,us,Amazon,10.0,THE BEST NEXUS EVER,Amazon Customer,"Huawei Nexus 6P unlocked smartphone, 32GB Gold..."
999994,12/9/2009,en,us,Amazon,2.0,Worst phone I've ever had,Amazon Customer,Casio G'zOne Commando 4G LTE C811 Verizon Andr...
999996,12/27/2013,cs,cz,Alza,9.0,"Telefon zakoupen pro starÅ¡Ã­ho ÄlovÄka ,kte...",ZÃ¡kaznÃ­k,Nokia 108


Crerating a list of authors who passed our criteria for filtering : 

In [26]:
counts_a = (counts[counts>=50].index)
counts_a

Index(['Amazon Customer', 'ÐÐ»ÐµÐºÑÐ°Ð½Ð´Ñ', 'Cliente Amazon',
       'Amazon Kunde', 'e-bit', 'Client d'Amazon', 'ÐÐ¾ÑÑÑ', 'unknown',
       'Anonymous', 'einer Kundin',
       ...
       'Sean', 'mrgreg', 'Jeremy', 'nicola', 'Kate', '???????????? ??????????',
       '???????????????? ??????????????', 'Anoniem', 'Manuela', 'Christoph'],
      dtype='object', length=669)

Selecting products with 50 or 50+ reviews :

In [27]:
count_p = df['product'].value_counts()
df_p = df[df['product'].isin(count_p[count_p>=50].index)]
df_p

,date,lang,country,source,score,extract,author,product
1,5/26/2016,fi,fi,Verkkokauppa,8.0,Alle 200â¬:n Ã¤lypuhelimeksi Huawei P8 Lite o...,Tekido81,"Huawei P8 Lite Dual-SIM âAndroid-puhelin, musta"
4,2/4/2014,de,de,Amazon,10.0,"Ich habe dieses Mobile Phone zweimal gekauft, ...",BÃ¼chermania,"Samsung E1150i Klapphandy 3,6 cm (1,43 Zoll) D..."
5,11/4/2013,ru,ru,Yandex,2.0,?????????????????? ??????????? ????????? ???...,ÐÐ»ÐµÐºÑÐ°Ð½Ð´Ñ,HTC Sensation XE
6,1/2/2016,nl,be,KIESKEURIG,8.0,De Huawei P8 Lite is een goed mooi toestel dat...,Jesse van Leeuwen,Huawei P8lite zwart / 16 GB
7,10/19/2016,en,us,Amazon,2.0,Do NOT buy this phone! You are better off gett...,Leo,"BLU Vivo XL Smartphone - 5.5"" 4G LTE - GSM Unl..."
...,...,...,...,...,...,...,...,...
999993,2/5/2016,en,us,LG Electronics,10.0,I have owned many phones from galaxy s2 to s6 ...,Beltran091387,LG None None
999994,12/9/2009,en,us,Amazon,2.0,Worst phone I've ever had,Amazon Customer,Casio G'zOne Commando 4G LTE C811 Verizon Andr...
999995,7/16/2016,de,de,Amazon,10.0,Da das Nexus 5X jetzt in realistische Preisreg...,Duke,"LG Electronics Nexus 5X Google Smartphone (13,..."
999997,5/18/2016,en,in,Amazon,10.0,Flawless delivery of a flawless phone!,Prashant Kashyap,"Apple iPhone SE (Gold, 16GB)"


Crerating a list of products who passed our criteria for filtering : 

In [28]:
counts_p = (count_p[count_p>=50].index)
counts_p

Index(['Lenovo Vibe K4 Note (White,16GB)', 'Lenovo Vibe K4 Note (Black, 16GB)',
       'OnePlus 3 (Graphite, 64 GB)', 'OnePlus 3 (Soft Gold, 64 GB)',
       'Samsung Galaxy Express I8730', 'Huawei P8lite zwart / 16 GB',
       'Lenovo Vibe K5 (Gold, VoLTE update)',
       'Samsung Galaxy S6 zwart / 32 GB',
       'Lenovo Vibe K5 (Grey, VoLTE update)', 'Nokia 5800 XpressMusic',
       ...
       'Blackberry Bold 9790 Smartphone 8GB (6,4 cm (2,5 Zoll) Touchscreen, 5 Megapixel Kamera, QWERTZ) schwarz',
       'Sony Xperia SP C5302 Unlocked Phone--U.S. Warranty (Black)',
       'Sony Ericsson Spiro Handy (3.5mm Klinkenstecker, TrackID, 2.0 MP, UKW-Radio) stealth',
       'Alcatel OneTouch IDOL X 6040D',
       'Huawei P8 lite Smartphone, Display 5,0 pollici IPS, Dual sim, Processore Octa-Core, 16 GB, Fotocamera 13 MP, Bianco',
       'Lenovo S856 - Smartphone Libre 4G Android 4.4 (Quad Core, Ram 1GB, Rom 8GB, 5.5'' Pantalla 1280*720 Pixels, CÃ¡mara 8Mp, Dual SIM, Wifi, Gps) (Dorado)',
    

Filtering dataframe with authors who have gave 50+ reviews and products which got 50+ reviews

In [29]:
df_final = df[(df['author'].isin(counts_a)) & (df['product'].isin(counts_p))] 
df_final.sample(5)

,date,lang,country,source,score,extract,author,product
497213,11/12/2015,it,it,Amazon,2.0,Il prodotto Ã¨ arrivato brandizzato da un oper...,Cliente Amazon,"LG G2 Smartphone (13,2 cm (5,2 pollici), Touch..."
979776,11/9/2014,en,us,Amazon,10.0,Is a fantastic phone!! i'm very happy. Super f...,David,"SONY XPERIA Z3 D6603 LTE 16GB 5.2"" GSM Unlocke..."
450756,11/10/2013,de,de,Ciao,10.0,Wirft man einen Blick auf die Produktspezifika...,Qantas,Nokia Lumia 1020
511431,2/6/2010,tr,tr,Tiklayaz,8.0,.Ã¶zellikle orta yaÅ grubuna hitap eder cinst...,black,Samsung M620 Cep Telefonu
741216,12/16/2010,ru,ru,Yandex,8.0,"Ð ÑÐµÐ»Ð¾Ð¼ Ð¾ÑÐ»Ð¸ÑÐ½ÑÐ¹ ÑÐµÐ»ÐµÑÐ¾Ð½,...",ÐÐ»ÐµÐºÑÐ°Ð½Ð´Ñ,Nokia 5230


Now we will see, after filtering how many unique authors and products are present in our filtered dataframe

Checking the unique count for Authors and Products in Raw Dataframe vs Filtered Dataframe

In [30]:
print('Unique Authors in Dataset with 1000000 samples : ',df['author'].nunique())
print('\nUnique Product in Dataset with 1000000 samples : ',df['product'].nunique())
print('\nUnique Authors in Dataset with products and authors who have 50 or more than 50 records : ',df_final['author'].nunique())
print('\nUnique Product in Dataset with products and authors who have 50 or more than 50 records : ',df_final['product'].nunique())

Unique Authors in Dataset with 1000000 samples :  596424

Unique Product in Dataset with 1000000 samples :  56448

Unique Authors in Dataset with products and authors who have 50 or more than 50 records :  667

Unique Product in Dataset with products and authors who have 50 or more than 50 records :  4158




1.   We can see that considerable amount of authors have less than 50 reviews as unique author count is very less compared to our original one.
2.   For products it is roughly 8 percent of the total products in our pool.



Creating a new feature No. of reviews for better understanding of popularity for creating Popularity Based Model:

In [31]:
data = df_final.groupby('product').agg({'author':'count'}).reset_index()
data.rename(columns = {'author':'No. of Reviews'}, inplace = True)
data.sort_values(['No. of Reviews','product'], ascending= False)

,product,No. of Reviews
1564,"Lenovo Vibe K4 Note (White,16GB)",2228
1563,"Lenovo Vibe K4 Note (Black, 16GB)",1806
2440,"OnePlus 3 (Graphite, 64 GB)",1330
2441,"OnePlus 3 (Soft Gold, 64 GB)",1219
1565,"Lenovo Vibe K5 (Gold, VoLTE update)",1112
...,...,...
162,Apple iPhone 5,1
136,Apple iPhone 4S 16GB,1
49,Alcatel One Touch 511,1
2,6682 / 6681,1


Top 5 mobile phones :

In [32]:
data_sorted = data.sort_values(['No. of Reviews','product'], ascending = False)
data_sorted['Rank'] = data_sorted['No. of Reviews'].rank(ascending = 0, method ='dense')
data_sorted.head(5)

,product,No. of Reviews,Rank
1564,"Lenovo Vibe K4 Note (White,16GB)",2228,1.0
1563,"Lenovo Vibe K4 Note (Black, 16GB)",1806,2.0
2440,"OnePlus 3 (Graphite, 64 GB)",1330,3.0
2441,"OnePlus 3 (Soft Gold, 64 GB)",1219,4.0
1565,"Lenovo Vibe K5 (Gold, VoLTE update)",1112,5.0


**These are the top 5 Products based on a simple Popularity Based Model**

1.   Converting Pandas DataFrame into Surprise DataFrame.
2.   Splitting into Train, Test for further proceeding.

1.   Checking Length of Train and Test Set. 




In [33]:
reader = Reader(rating_scale=(0,10))
data = Dataset.load_from_df(df_final[['author','product','score']], reader)
trainset,testset = train_test_split(data, test_size=0.25, random_state=612)
print('Train Set Length :',trainset.n_ratings)
print('Test  Set Lenght :',len(testset))

Train Set Length : 98158
Test  Set Lenght : 32720


Building Collaborative Filtering Model(Using SVD) :


In [34]:
svd_model = SVD(random_state = 612)
svd_model.fit(trainset)
train_pred = svd_model.test(trainset.build_testset())
pred = svd_model.test(testset)
print('Train Set RMSE ',accuracy.rmse(train_pred))
print('Test  Set RMSE ',accuracy.rmse(pred))

RMSE: 2.3101
Train Set RMSE  2.310076640075207
RMSE: 2.6070
Test  Set RMSE  2.607033037678817


We are getting a righ RMSE score on test set in contrast with our train set. 

Calculating error by subtracting original value from predicted value, and observing best and worst predictions by our model based on that error :  

In [35]:
pred_df = pd.DataFrame(pred)
pred_df['err'] = abs(pred_df.est - pred_df.r_ui)
best_predictions = pred_df.sort_values(by='err')[:10]
best_predictions

,uid,iid,r_ui,est,details,err
11979,Ð¡ÐµÑÐ³ÐµÐ¹,Samsung Galaxy A3,10.0,10.0,{'was_impossible': False},0.0
27688,ÐÐ»ÐµÐ½Ð°,Samsung Galaxy Note5,10.0,10.0,{'was_impossible': False},0.0
856,rick2809,Smartphone Apple iPhone 6 16GB,10.0,10.0,{'was_impossible': False},0.0
27101,ÐÐ¸Ñ Ð°Ð¸Ð»,Samsung Galaxy Note5,10.0,10.0,{'was_impossible': False},0.0
23140,luigi,"Lenovo Motorola Moto G Smartphone, 4,5 pollici...",10.0,10.0,{'was_impossible': False},0.0
21713,Ð¢Ð°ÑÑÑÐ½Ð°,Samsung Galaxy S7 edge,10.0,10.0,{'was_impossible': False},0.0
23153,Gianluca,"Lenovo Motorola Moto G Smartphone, 4,5 pollici...",10.0,10.0,{'was_impossible': False},0.0
24787,Davide,"Alcatel One Touch Pop C5 Smartphone, Dual SIM,...",10.0,10.0,{'was_impossible': False},0.0
7690,Gianluca,"Lenovo Motorola Moto G Smartphone, Display HD ...",10.0,10.0,{'was_impossible': False},0.0
17022,Andrea,"Samsung G920 Galaxy S6 Smartphone, 32 GB, Oro ...",10.0,10.0,{'was_impossible': False},0.0


In [36]:
worst_predictions = pred_df.sort_values(by='err')[-10:]
worst_predictions

,uid,iid,r_ui,est,details,err
25157,Kindle Customer,Samsung Galaxy Grand Prime Dual Sim Factory Un...,2.0,9.938796,{'was_impossible': False},7.938796
11922,Andrew,"APPLE iPhone 7 Plus - Black, 128 GB",1.0,8.992049,{'was_impossible': False},7.992049
2916,Maurizio,"Asus ZenFone 3 Smartphone, Memoria Interna da ...",2.0,10.000000,{'was_impossible': False},8.000000
9338,Amazon Customer,"Blu Win JR LTE (Grey, 4GB)",2.0,10.000000,{'was_impossible': False},8.000000
15776,marco,"Asus ZenFone 2 Laser Smartphone, Display da 5""...",2.0,10.000000,{'was_impossible': False},8.000000
11652,Francesco,"LG Nexus 4 Smartphone, Nero [Italia]",2.0,10.000000,{'was_impossible': False},8.000000
23070,Amazon Customer,"Blu Win JR LTE (Grey, 4GB)",2.0,10.000000,{'was_impossible': False},8.000000
528,David,LG G2 - Smartphone libre Android (pantalla 5.2...,2.0,10.000000,{'was_impossible': False},8.000000
2987,Mark,"APPLE iPhone 7 Plus - Black, 128 GB",1.0,9.008105,{'was_impossible': False},8.008105
30298,Chris,"Lenovo Motorola Defy+ Smartphone (9,3 cm (3,7 ...",10.0,1.212229,{'was_impossible': False},8.787771


So we can see that are SVD model is performing quite well for the high score, but in our worst predictions we can see it is giving score 9/10 to products who have got only 1/2 score. For products having 10 as primal rating our model is good in predicting them.

Building Collaborative Model(Using KNNwithMeans) :

In [37]:
knn_msd_model = KNNWithMeans(random_state = 612,k = 50,sim_options = {'user_based': True})
knn_msd_model.fit(trainset)
knn_msd_train_pred = knn_msd_model.test(trainset.build_testset())
knn_msd_pred = knn_msd_model.test(testset)
print('Train Set RMSE ',accuracy.rmse(knn_msd_train_pred))
print('Test  Set RMSE ',accuracy.rmse(knn_msd_pred))

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 2.2634
Train Set RMSE  2.263385257753667
RMSE: 2.6233
Test  Set RMSE  2.623266410072608


In [38]:
knn_cos_model = KNNWithMeans(random_state = 612,k = 50,sim_options = {'name': 'cosine'})
knn_cos_model.fit(trainset)
knn_cos_train_pred = knn_cos_model.test(trainset.build_testset())
knn_cos_pred = knn_cos_model.test(testset)
print('Train Set RMSE ',accuracy.rmse(knn_cos_train_pred))
print('Test  Set RMSE ',accuracy.rmse(knn_cos_pred))

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 2.4542
Train Set RMSE  2.4541877329389576
RMSE: 2.5909
Test  Set RMSE  2.5909283978783013


We are trying 2 models with KNNWithMeans :


1.   User based KNN model with 50 as k value and using MSD similarity matrix.
2.   User based KNN model with 50 as k value and using Cosine similarity matrix.

We observed that MSD one is giving better results for train set but in test set it is not performing that well as cosine one is performing.

For item based tried with 10000 samples too, it is not able to execute on my system, for some reason my colab crashing again and again.




*   **Moving on with KNN with cosine similarity matrix for prediction of scores for our test users as it has less RMSE for test data.**
*   **Observing best and worst predictions made by our model for sample users.**



In [39]:
knn_cos_pred_df = pd.DataFrame(knn_cos_pred)
knn_cos_pred_df['err'] = abs(knn_cos_pred_df.est - knn_cos_pred_df.r_ui)
knn_cos_best_predictions = knn_cos_pred_df.sort_values(by='err')[:10]
knn_cos_best_predictions

,uid,iid,r_ui,est,details,err
8725,okuyan,Apple iphone 3G 16GB Cep Telefonu,8.0,8.0,"{'actual_k': 33, 'was_impossible': False}",0.0
441,dolunay,Nokia N81 Cep Telefonu,8.0,8.0,"{'actual_k': 40, 'was_impossible': False}",0.0
21269,Aksisey,Nokia 6500 Slide Cep Telefonu,8.0,8.0,"{'actual_k': 36, 'was_impossible': False}",0.0
29270,Kelly,"HTC 8X Unlocked Smartphone C620E, 16 GB, Blue",10.0,10.0,"{'actual_k': 3, 'was_impossible': False}",0.0
19238,massimo,"Samsung Galaxy A5 2016 Smartphone LTE, 16GB, Nero",10.0,10.0,"{'actual_k': 50, 'was_impossible': False}",0.0
3633,massimo,"Honor 7 Smartphone 4G, Display Full HD 5.2 Pol...",10.0,10.0,"{'actual_k': 50, 'was_impossible': False}",0.0
15451,Tony,Sim Free Samsung Galaxy S6 Edge Mobile Phone -...,10.0,10.0,"{'actual_k': 1, 'was_impossible': False}",0.0
30729,miatamania,Nokia 8800 Arte Cep Telefonu,8.0,8.0,"{'actual_k': 45, 'was_impossible': False}",0.0
7081,Jack,LG Nexus 5 D820 16GB Unlocked GSM 4G LTE Quad-...,10.0,10.0,"{'actual_k': 5, 'was_impossible': False}",0.0
19249,karol07,Nokia N81 Cep Telefonu,8.0,8.0,"{'actual_k': 42, 'was_impossible': False}",0.0


In [40]:
knn_cos_worst_predictions = knn_cos_pred_df.sort_values(by='err')[-10:]
knn_cos_worst_predictions

,uid,iid,r_ui,est,details,err
9155,JD,BLU Zoey II Quadband Unlocked Dual Sim Phone w...,2.0,10.000000,"{'actual_k': 14, 'was_impossible': False}",8.000000
22518,Chris,Wileyfox Swift 4G Dual SIM-Free Smartphone - W...,2.0,10.000000,"{'actual_k': 16, 'was_impossible': False}",8.000000
5131,Frank,Samsung Galaxy S II i9100 DualCore Smartphone ...,2.0,10.000000,"{'actual_k': 3, 'was_impossible': False}",8.000000
9576,Carlo,Motorola Moto G 5-Inch (2nd Gen UK Stock) Dual...,2.0,10.000000,"{'actual_k': 9, 'was_impossible': False}",8.000000
11812,Kindle Customer,BlackBerry Bold 9700 Sim Free Smartphone - Black,10.0,1.970595,"{'actual_k': 2, 'was_impossible': False}",8.029405
27964,Roland,Motorola V360,1.0,9.056197,"{'actual_k': 11, 'was_impossible': False}",8.056197
1414,Amazon Customer,BlackBerry Bold 9700 Sim Free Smartphone - Black,10.0,1.835251,"{'actual_k': 2, 'was_impossible': False}",8.164749
26746,Amazon Customer,Nokia 225 UK SIM-Free Mobile Phone - Black,10.0,1.560457,"{'actual_k': 2, 'was_impossible': False}",8.439543
21507,eduardo,Nokia 500,10.0,1.320320,"{'actual_k': 3, 'was_impossible': False}",8.679680
13084,Gast,"Blackberry Bold 9900 Smartphone (7,1 cm (2,8 Z...",10.0,0.099279,"{'actual_k': 4, 'was_impossible': False}",9.900721






*   Here we can see a clear differnce from SVD that KNN with cosine similarity one is not only predicting higher values for the original low ones but also predicting less values for the original high ones. 
*   Moreover in the SVD best predictions we are only able to see top rated as top predicted, but here in KNN both predictions we have varying scores, that is a good sign that our model is understanding more information and working better with information gain. 




In [41]:
def get_top_n(predictions, id, n):
    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n[id]

In [42]:
get_top_n(knn_cos_pred,'Daniele',5)

[('Lenovo Motorola Moto G LTE Smartphone, Display 4.5", Fotocamera 5 MP, Memoria 8 GB, Quad-Core 1.2 GHz, 1 GB RAM, Micro SD, Android 4.4.3, Bianco [Italia]',
  10),
 ("Huawei P9 Lite Smartphone, LTE, Display 5.2'' FHD, Processore Octa-Core Kirin 650, 16 GB Memoria Interna, 3GB RAM, Fotocamera 13 MP, Single-SIM, Android 6.0 Marshmallow, Bianco [Italia]",
  10),
 ('Samsung Galaxy S3 mini I8190 Smartphone, Display AMOLED da 10.2 cm (4 Pollici), Dual Core, 1 GHz, 1 GB RAM, Fotocamera 5 Megapixel, Wi-Fi, Android 4.1, Blu [Germania]',
  9.97314094621141),
 ('elephone P8000 Smartphone 4G FDD-LTE 64bit MTK6753 Otta Nucleo FHD 5.5 "IPS Android 5.1 3GB+16GB 5MP 13mp Due telecamere il tocco a 360 Â° ID imporonta digitale',
  9.737251039064141),
 ('LG D802 G2 Smartphone, 16 GB, Nero [Italia]', 9.729460603492774)]

In [43]:
get_top_n(knn_cos_pred,'Francesco',5)

[('Honor 6X Smartphone 4G LTE, Diplay 5.5 pollici FHD, 32 GB ROM, 3 GB RAM, Dual Camera 12 Megapixel, Sensore Fingerprint, Android, Grigio',
  10),
 ('Lenovo Motorola Moto G Smartphone, 4,5 pollici display HD, processore Qualcomm, memoria 16GB, MicroSIM, Android 4.3 OS, fotocamera da 5 MP, Nero [Germania]',
  9.801463453273074),
 ('Lenovo Motorola Moto G Smartphone, 4,5 pollici display HD, processore Qualcomm, memoria 16GB, MicroSIM, Android 4.3 OS, fotocamera da 5 MP, Nero [Germania]',
  9.801463453273074),
 ('Lenovo Motorola Moto G LTE Smartphone, Display 4.5", Fotocamera 5 MP, Memoria 8 GB, Quad-Core 1.2 GHz, 1 GB RAM, Micro SD, Android 4.4.3, Bianco [Italia]',
  9.779880524651418),
 ('Lenovo Motorola Moto G LTE Smartphone, Display 4.5", Fotocamera 5 MP, Memoria 8 GB, Quad-Core 1.2 GHz, 1 GB RAM, Micro SD, Android 4.4.3, Bianco [Italia]',
  9.779880524651418)]

**These both are the Top 5 recommendations for our different test users.**

Cross Validating our 3 models with different fold values :

In [44]:
from surprise.model_selection import cross_validate
models = []
models.append(('SVD', svd_model))
models.append(('KNN - MSD', knn_msd_model))
models.append(('KNN - Cosine', knn_cos_model))

In [45]:
scores = []

# Iterate over all algorithms
for name,algorithm in models:
    # Perform cross validation
    results = cross_validate(algorithm, data, measures=['RMSE','MAE'], cv=7, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series(name, index=['Algorithm']))
    scores.append(tmp)

pd.DataFrame(scores).set_index('Algorithm').sort_values('test_rmse')  

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity m

,test_rmse,test_mae,fit_time,test_time
Algorithm,,,,
KNN - Cosine,2.585995,2.011799,0.588004,2.432421
KNN - MSD,2.610031,2.011842,0.414778,2.461595
SVD,2.612054,2.028310,5.582962,0.126132


After cross - validating :


1.   KNN - Cosine have the least RMSE score on test set.
2.   KNN - Cosine have the least MEA score on test set.
1.   Both KNN technique take less time in fitting and spend more time in testing data which is a good scope to use KNN in production models.
2.   SVD takes more time in fitting the data rather testing, also it is taking comparatively more time then both KNN together, so if we have to work with load of dataset and want out iteration to be fast, then SVD is not our cuppa tea.





*In what business scenario you should use popularity based Recommendation Systems?*

**Business which are not developed yet, i.e, new business as they do not have any prior data about users on their site, they should use popularity based as it will help them in attracting customers to their business.**
**If any new service is being launched or any new demographic location is added to our business, we should pick up popularity based recommender system as to make our customer base strong and after that we can move on with collaborative filtering.**

*In what business scenario you should use CF based Recommendation Systems?*

**If our business have data of user traffic then we can use CF recommender sytem, like Netflix,Spotify,Amazon do have. As i remember there was movies/music/prodcuts recommended to me earlier too but they were not on point but as days passes it got a strong recommendation for me, because now they have more data points on me as they did back earlier. So, all this Data hungry algorithm can provide a good result if they have data, if they don't have data then there is no use of CF recommendation system. In that case, popularity based can provide better results.**

*What other possible methods can you think of which can further improve the recommendation for different users?*




*   **Sparsity and Scalability could be tackled more efficiently.**
*   **Regarding this dataset, for many languages we have product names and author names which are in different language, that can be regularized with only one language.**


*   **Size of the dataset, as it will enhance our chances of good recommendations. Subset of a very large dataset which captures all the raw features of user-item, can make our model's information gain much higher.** 





In [47]:
%%shell
jupyter nbconvert --to html /content/Recommendation_System_R6_Project_Mobile_Reviews.ipynb

[NbConvertApp] Converting notebook /content/Recommendation_System_R6_Project_Mobile_Reviews.ipynb to html
[NbConvertApp] Writing 402375 bytes to /content/Recommendation_System_R6_Project_Mobile_Reviews.html
